# モデルのトレーニング (YOLOv8 Ultralytics)

このノートブックは、準備されたデータセット (`data.yaml` を使用) を用いて、Ultralytics YOLOv8モデルをトレーニングします。

**主なステップ:**
1.  **必要なライブラリのインポート:** `ultralytics` をインポートします。
2.  **設定の定義:** `data.yaml` ファイルのパス、使用するYOLOv8モデルのバリアント (例: `yolov8n.pt`)、エポック数、画像サイズなどを設定します。
3.  **モデルのトレーニング:** YOLOオブジェクトの `train()` メソッドを呼び出してトレーニングを実行します。
4.  **結果の確認:** トレーニング結果 (重みファイルやログなど) は、通常 `runs/detect/train` のようなディレクトリに保存されます。

In [1]:
from ultralytics import YOLO
import os
import torch # PyTorchのインポートも念のため残しておきます

## 1. 設定

In [4]:
# --- トレーニング設定 ---

# data.yaml ファイルへのパス
# 前のノートブック (3_dataset_preparation_for_yolo.ipynb) で生成されたものを指定します。
# 例: '../data/processed/tennis_dataset_yolo_prepared/data.yaml'
DATA_YAML_PATH = '../data/processed/dataset/data.yaml' 

# 使用するYOLOv8モデルの事前学習済みウェイト
# 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt' など
# または、カスタムモデルの場合は .pt ファイルへのパス
MODEL_VARIANT = 'yolov8s.pt'

# トレーニングのエポック数
NUM_EPOCHS = 50

# トレーニング時の入力画像サイズ
IMAGE_SIZE = 1280

# バッチサイズ
BATCH_SIZE = 16 # GPUメモリに応じて調整してください

# トレーニング結果が保存されるプロジェクト名 (runs/detect/PROJECT_NAME になる)
PROJECT_NAME = 'tennis_detection_project'
# トレーニング実行名 (runs/detect/PROJECT_NAME/RUN_NAME になる)
RUN_NAME = 'exp1'

# GPUを使用するかどうか (利用可能であれば自動的に使用されます)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"使用デバイス: {DEVICE}")

# data.yaml の存在確認
if not os.path.exists(DATA_YAML_PATH):
    print(f"警告: data.yaml が見つかりません: {os.path.abspath(DATA_YAML_PATH)}")
    print("前のノートブックで data.yaml を生成したか、パスが正しいか確認してください。")
else:
    print(f"data.yaml を使用します: {os.path.abspath(DATA_YAML_PATH)}")

使用デバイス: cuda
data.yaml を使用します: c:\Users\akama\AppData\Local\Programs\Python\Python310\python_file\projects\tennisvision\data\processed\dataset\data.yaml


## 2. YOLOv8モデルの初期化とトレーニング

In [7]:
# YOLOモデルをロード (事前学習済みウェイトを使用)
# MODEL_VARIANT が .pt ファイルへのパスでない場合、Ultralyticsが自動的にダウンロードします。
model = YOLO(MODEL_VARIANT)

# モデルをトレーニング
if os.path.exists(DATA_YAML_PATH):
    print("\nトレーニングを開始します...")
    try:
        results = model.train(
            data=DATA_YAML_PATH,
            epochs=NUM_EPOCHS,
            imgsz=IMAGE_SIZE,
            batch=BATCH_SIZE,
            project=PROJECT_NAME,
            name=RUN_NAME,
            device=DEVICE, # 明示的にデバイスを指定
            # patience=10, # Early stopping の設定 (オプション)
            # workers=8,   # データローダーのワーカー数 (オプション)
            # verbose=True # 詳細なログ出力 (デフォルトはTrue)
        )
        print("\nトレーニングが完了しました。")
        print(f"結果は {results.save_dir} に保存されました。")
        print(f"最適な重みは {results.best} に保存されています。")
    except Exception as e:
        print(f"トレーニング中にエラーが発生しました: {e}")
        import traceback
        traceback.print_exc()
else:
    print("data.yaml が見つからないため、トレーニングをスキップします。パスを確認してください。")


トレーニングを開始します...
New https://pypi.org/project/ultralytics/8.3.140 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.139  Python-3.10.11 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/processed/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp13, nbs=64, nm

100%|██████████| 755k/755k [00:00<00:00, 11.8MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

 12                  -1  1    591360  ultralytics.nn.modules.block.C2f             [768, 256, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 16                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 18                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 19                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

100%|██████████| 5.35M/5.35M [00:00<00:00, 43.5MB/s]


AMP: checks passed 
トレーニング中にエラーが発生しました: train: Error loading data from C:\Users\akama\AppData\Local\Programs\Python\Python310\python_file\projects\tennisvision\data\processed\dataset\train\images
See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Traceback (most recent call last):
  File "c:\Users\akama\AppData\Local\Programs\Python\Python310\python_file\projects\tennisvision\venv\lib\site-packages\ultralytics\data\base.py", line 175, in get_img_files
    assert im_files, f"{self.prefix}No images found in {img_path}. {FORMATS_HELP_MSG}"
AssertionError: train: No images found in C:\Users\akama\AppData\Local\Programs\Python\Python310\python_file\projects\tennisvision\data\processed\dataset\train\images. Supported formats are:
images: {'webp', 'heic', 'jpg', 'tif', 'png', 'dng', 'pfm', 'jpeg', 'tiff', 'bmp', 'mpo'}
videos: {'mov', 'mpeg', 'asf', 'avi', 'mpg', 'mkv', 'wmv', 'mp4', 'ts', 'webm', 'gif', 'm4v'}

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\akama\AppData\Local\Temp\ipykernel_11328\3299308150.py", line 9, in <module>
    results = model.train(
  File "c:\Users\akama\AppData\Local\Programs\Python\Python310\python_file\projects\tennisvision\venv\

## 3. トレーニング結果の確認と次のステップ

トレーニングが正常に完了すると、結果は `runs/detect/<PROJECT_NAME>/<RUN_NAME>` ディレクトリ (例: `runs/detect/tennis_detection_project/exp1`) に保存されます。このディレクトリには以下のものが含まれます:

*   **重みファイル (`weights/`):**
    *   `best.pt`: 検証セットで最良のパフォーマンスを示したモデルの重み。
    *   `last.pt`: 最後のエポック完了時のモデルの重み。
*   **結果のプロット:**
    *   `results.png`: 損失やメトリクス (mAPなど) のグラフ。
    *   `confusion_matrix.png`: 混同行列。
    *   その他、検証バッチの予測結果の画像など。
*   **ログファイル:**
    *   `results.csv`: 各エポックのメトリクスが記録されたCSVファイル。
    *   `args.yaml`: トレーニングに使用された設定。

**次のステップ:**
1.  **結果の評価:** `results.png` や `results.csv` を確認して、モデルの学習進捗や最終的なパフォーマンスを評価します。
2.  **推論:** 保存された重みファイル (`best.pt` など) を使用して、新しい画像や動画で物体検出を行います。
    ```python
    # from ultralytics import YOLO
    # model = YOLO('runs/detect/tennis_detection_project/exp1/weights/best.pt')
    # results = model.predict(source='path/to/your/image_or_video.mp4', save=True)
    ```
3.  **ハイパーパラメータ調整:** 必要に応じて、エポック数、学習率 (YOLOv8では通常自動調整)、バッチサイズ、モデルアーキテクチャなどを変更して再トレーニングを行います。

**注意:**
*   `DATA_YAML_PATH` が正しく設定されていることを確認してください。
*   GPUメモリが不足する場合は、`BATCH_SIZE` を小さくするか、`IMAGE_SIZE` を小さくしてみてください。
*   トレーニングには時間がかかることがあります。